In [1]:
from google.colab import drive
drive.mount('/content/drive')
!pip install kaggle


Mounted at /content/drive


In [2]:
from google.colab import files

uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
import os

os.makedirs('/root/.kaggle', exist_ok=True)
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [4]:
!kaggle datasets download -d kmader/skin-cancer-mnist-ham10000 -p /content/

Dataset URL: https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000
License(s): CC-BY-NC-SA-4.0
100% 5.19G/5.20G [00:51<00:00, 60.0MB/s]
100% 5.20G/5.20G [00:51<00:00, 109MB/s] 


In [5]:
import zipfile

with zipfile.ZipFile('/content/skin-cancer-mnist-ham10000.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/ham10000')

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [7]:
import pandas as pd
from pathlib import Path
csv_path = "/content/ham10000/HAM10000_metadata.csv"
skin_df = pd.read_csv(csv_path)
skin_df.sort_values(by="image_id")

,lesion_id,image_id,dx,dx_type,age,sex,localization
4349,HAM_0000550,ISIC_0024306,nv,follow_up,45.0,male,trunk
4263,HAM_0003577,ISIC_0024307,nv,follow_up,50.0,male,lower extremity
4217,HAM_0001477,ISIC_0024308,nv,follow_up,55.0,female,trunk
3587,HAM_0000484,ISIC_0024309,nv,follow_up,40.0,male,trunk
1451,HAM_0003350,ISIC_0024310,mel,histo,60.0,male,chest
...,...,...,...,...,...,...,...
1721,HAM_0004304,ISIC_0034316,mel,histo,85.0,male,upper extremity
1888,HAM_0006376,ISIC_0034317,mel,histo,70.0,female,lower extremity
121,HAM_0000344,ISIC_0034318,bkl,histo,55.0,male,trunk
7440,HAM_0000747,ISIC_0034319,nv,histo,30.0,male,trunk


In [8]:
path = Path('../content/ham10000')
Path.BASE_PATH = path

In [9]:
short_to_full_name_dict = {
    "akiec" : "Bowen's disease", # very early form of skin cancer
    "bcc" : "basal cell carcinoma" , # basal-cell cancer or white skin cancer
    "bkl" : "benign keratosis-like lesions", # non-cancerous skin tumour
    "df" : "dermatofibroma", # non-cancerous rounded bumps
    "mel" : "melanoma", # black skin cancer
    "nv" : "melanocytic nevi", # mole non-cancerous
    "vasc" : "vascular lesions", # skin condition
}

In [10]:
img_to_class_dict = skin_df.loc[:, ["image_id", "dx"]]
img_to_class_dict = img_to_class_dict.to_dict('list')
img_to_class_dict = {img_id : short_to_full_name_dict[disease] for img_id,disease in zip(img_to_class_dict['image_id'], img_to_class_dict['dx']) }
[x for x in img_to_class_dict.items()][:5]

[('ISIC_0027419', 'benign keratosis-like lesions'),
 ('ISIC_0025030', 'benign keratosis-like lesions'),
 ('ISIC_0026769', 'benign keratosis-like lesions'),
 ('ISIC_0025661', 'benign keratosis-like lesions'),
 ('ISIC_0031633', 'benign keratosis-like lesions')]

In [11]:
def get_label_from_dict(path):
    return img_to_class_dict[path.stem]



In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from typing import Type, Tuple

In [13]:
class BasicBlock(nn.Module):
  def __init__(
        self,
        in_channels: int,
        out_channels: int,
        stride: int = 1,
        expansion: int = 1,
        downsample: nn.Module = None
    ) -> None:
        super(BasicBlock, self).__init__()
        self.expansion = expansion
        self.downsample = downsample
        self.conv1 = nn.Conv2d(
            in_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels*self.expansion,
            kernel_size=3,
            padding=1,
            bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels*self.expansion)
  def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out






In [14]:
class ResNet(nn.Module):
    def __init__(
        self,
        img_channels: int,
        num_layers: int,
        block: Type[BasicBlock],
        num_classes: int = 1000
    ) -> None:
        super(ResNet, self).__init__()
        if num_layers == 18:
            layers = [2, 2, 2, 2]
            self.expansion = 1

        self.in_channels = 64
        self.conv1 = nn.Conv2d(
            in_channels=img_channels,
            out_channels=self.in_channels,
            kernel_size=7,
            stride=2,
            padding=3,
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(self.in_channels)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512*self.expansion, num_classes)

    def _make_layer(
        self,
        block: Type[BasicBlock],
        out_channels: int,
        blocks: int,
        stride: int = 1
    ) -> nn.Sequential:
        downsample = None
        if stride != 1:
            downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels*self.expansion,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * self.expansion),
            )
        layers = []
        layers.append(
            block(
                self.in_channels, out_channels, stride, self.expansion, downsample
            )
        )
        self.in_channels = out_channels * self.expansion

        for _ in range(1, blocks):
            layers.append(block(
                self.in_channels,
                out_channels,
                expansion=self.expansion
            ))
        return nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [15]:
from sklearn.metrics import precision_score, recall_score, f1_score
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.metrics import precision_score, recall_score, f1_score
from typing import Tuple


def train(model: nn.Module, train_loader: DataLoader, optimizer: optim.Optimizer, criterion: nn.Module, device: torch.device) -> None:
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: [Batch {batch_idx}/{len(train_loader)} ({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

def test(model: nn.Module, test_loader: DataLoader, criterion: nn.Module, device: torch.device) -> Tuple[float, float]:
    model.eval()
    test_loss = 0
    correct = 0
    predictions = []
    targets = []

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += criterion(output, target).item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

            predictions.append(pred.cpu().numpy())
            targets.append(target.cpu().numpy())

    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)

    average_precision = precision_score(targets, predictions, average='weighted')
    average_recall = recall_score(targets, predictions, average='weighted')
    average_f1_score = f1_score(targets, predictions, average='weighted')

    print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({accuracy:.2f}%)\n')
    print(f"Average Precision: {average_precision:.2f}")
    print(f"Average Recall: {average_recall:.2f}")
    print(f"Average F1-score: {average_f1_score:.2f}")

    return test_loss, accuracy



In [17]:
    from torch import optim
    import torchvision.transforms as transforms
    import numpy as np

    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    ])

    train_dataset = datasets.ImageFolder("/content/ham10000", transform=transform)

    train_size = int(0.8 * len(train_dataset))
    val_size = len(train_dataset) - train_size
    train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

    train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=1)
    val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=1)

    model = ResNet(img_channels=3, num_layers=18, block=BasicBlock, num_classes=7)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)


    num_epochs = 30
    for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_loader, optimizer, criterion,device)
        val_loss, accuracy = test(model, val_loader, criterion,device)


Epoch 1/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 2.031519
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.361410
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.404167

Test set: Average loss: 0.0215, Accuracy: 1147/4006 (28.63%)

Average Precision: 0.30
Average Recall: 0.29
Average F1-score: 0.23
Epoch 2/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 1.383959
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.428681
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.361081

Test set: Average loss: 0.0218, Accuracy: 1096/4006 (27.36%)

Average Precision: 0.21
Average Recall: 0.27
Average F1-score: 0.23
Epoch 3/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Epoch: [Batch 0/251 (0%)]	Loss: 1.352439
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.382669
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.372034

Test set: Average loss: 0.0216, Accuracy: 1193/4006 (29.78%)

Average Precision: 0.37
Average Recall: 0.30
Average F1-score: 0.24
Epoch 4/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 1.319669
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.426578
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.383784

Test set: Average loss: 0.0215, Accuracy: 1180/4006 (29.46%)

Average Precision: 0.28
Average Recall: 0.29
Average F1-score: 0.22
Epoch 5/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 1.323411
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.385004
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.428645

Test set: Average loss: 0.0217, Accuracy: 1135/4006 (28.33%)

Average Precision: 0.14
Average Recall: 0.28
Average F1-score: 0.18
Epoch 6/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Epoch: [Batch 0/251 (0%)]	Loss: 1.334390
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.390359
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.420098

Test set: Average loss: 0.0215, Accuracy: 1161/4006 (28.98%)

Average Precision: 0.29
Average Recall: 0.29
Average F1-score: 0.23
Epoch 7/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 1.373499
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.364870
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.354002

Test set: Average loss: 0.0214, Accuracy: 1222/4006 (30.50%)

Average Precision: 0.25
Average Recall: 0.31
Average F1-score: 0.24
Epoch 8/30


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Epoch: [Batch 0/251 (0%)]	Loss: 1.362355
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.315881
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.377637

Test set: Average loss: 0.0213, Accuracy: 1159/4006 (28.93%)

Average Precision: 0.14
Average Recall: 0.29
Average F1-score: 0.19
Epoch 15/30


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Train Epoch: [Batch 0/251 (0%)]	Loss: 1.387870
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.320726
Train Epoch: [Batch 200/251 (80%)]	Loss: 1.310612

Test set: Average loss: 0.0213, Accuracy: 1155/4006 (28.83%)

Average Precision: 0.44
Average Recall: 0.29
Average F1-score: 0.19
Epoch 16/30
Train Epoch: [Batch 0/251 (0%)]	Loss: 1.312225
Train Epoch: [Batch 100/251 (40%)]	Loss: 1.270484


KeyboardInterrupt: 

In [ ]:
# Terminated the previous execution since accuracy isn't improving with epoch since the parameters are randomly initilized and is not pretrained (from-scratch implementation)